In [29]:
using Pkg
Pkg.add("CUDNN_jll")

   Resolving package versions...
    Updating `C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\Project.toml`
⌅ [62b44479] + CUDNN_jll v8.9.4+0
  No Changes to `C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\Manifest.toml`


In [30]:
using DataFrames
using Transformers
using Transformers.TextEncoders
using Transformers.HuggingFace
using TrillionDollarWords

### Load data

In [31]:
n = 10
data = load_training_sentences()
texts = filter(:split => n -> n == "test", data)[1:n, :]

Row,sentence,year,label,seed,sentence_splitting,event_type,split
,String,Int64,String7,Int64,Bool,String31,String7
1,"remained well below their levels at the beginning of the year, and that weaker demand and earlier declines in oil prices had been holding down consumer price inflation.",1996,hawkish,5768,true,meeting minutes,test
2,"A few participants also noted that uncertainty about the extent of resource slack in the economy was considerable and that it was quite possible that the economy could soon be operating close to potential, particularly if labor force participation rates did not turn up much while employment continued to register gains.",1996,neutral,5768,true,meeting minutes,test
3,inflation was projected to pick up gradually in association with a partial reversal of the decline in energy prices this year.,1996,neutral,5768,true,meeting minutes,test
4,They noted that the realization of such a development could make it harder for the Committee to achieve 2 percent inflation over the longer run.,1996,neutral,5768,true,meeting minutes,test
5,"In the view of one member, however, aggregate final demand was so strong that, with economic activity and the associated demand for labor having expanded at an unsustainable pace for some time, one could be reasonably confident that inflation would most likely pick up in the absence of policy action.",1996,hawkish,5768,true,meeting minutes,test
6,"In the circumstances, most members endorsed a proposal to delete as no longer necessary the previous summary statement relating to the risks to growth and inflation taken together.",1996,neutral,5768,true,meeting minutes,test
7,"In the staff forecast prepared for this meeting, the economy was seen as likely to expand at a moderate pace, supported by accommodative monetary policy and financial conditions.",1996,dovish,5768,true,meeting minutes,test
8,"Housing starts and the demand for new homes had declined further, house prices in many parts of the country were falling faster than they had towards the end of 2007, and inventories of unsold homes remained quite elevated.",1996,dovish,5768,true,meeting minutes,test
9,Pressures on resources would rise as the anticipated upturn and possible above-trend growth brought the economy closer to full capacity utilization.,1996,hawkish,5768,true,meeting minutes,test


### Get attributions

In [4]:
using CondaPkg
# CondaPkg.add("pytorch")
# CondaPkg.add("transformers"; version="4.15.0")
# CondaPkg.add("transformers-interpret")

    CondaPkg Found dependencies: C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\CondaPkg.toml
    CondaPkg Found dependencies: C:\Users\drobi\.julia\packages\PythonCall\wXfah\CondaPkg.toml
    CondaPkg Resolving changes
             + transformers-interpret
    CondaPkg Installing packages
             │ C:\Users\drobi\.julia\conda\3\x86_64\Scripts\mamba.exe
             │ install
             │ -y
             │ -p C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\.CondaPkg\env
             │ --override-channels
             │ --no-channel-priority
             │ python[version='>=3.8,<4']
             │ python[version='>=3.7,<4',channel='anaconda']
             │ pytorch[version='*']
             │ pytorch-cuda[version='12.1']
             │ transformers[version='4.15.0']
             │ transformers-interpret[version='*']
             │ -c anaconda
             │ -c conda-forge
             │ -c nvidia
         

anaconda/win-64                                             Using cache
anaconda/noarch                                             Using cache
conda-forge/win-64                                          Using cache
conda-forge/noarch                                          Using cache
nvidia/win-64                                               Using cache
nvidia/noarch                                               Using cache
pytorch/win-64                                              Using cache
pytorch/noarch                                              Using cache
anaconda/win-64                                             Using cache
anaconda/noarch                                             Using cache
Transaction

  Prefix: C:\Users\drobi\Desktop\uni\master_thesis\CounterfactualExplanations.jl\dev\notebooks\.CondaPkg\env

  Updating specs:

   - python[version='>=3.8,<4']
   - anaconda::python[version='>=3.7,<4']
   - pytorch=*
   - pytorch-cuda==12.1
   - transformers==4.15.0

In [5]:
using PythonCall

Precompiling PythonCall
  ✓ PythonCall
  1 dependency successfully precompiled in 20 seconds. 24 already precompiled.


In [12]:
transformers_interpret = PythonCall.pyimport("transformers_interpret")
transformers = PythonCall.pyimport("transformers")

Python: <module 'transformers' from 'C:\\Users\\drobi\\Desktop\\uni\\master_thesis\\CounterfactualExplanations.jl\\dev\\notebooks\\.CondaPkg\\env\\lib\\site-packages\\transformers\\__init__.py'>

In [13]:
# load pre-trained classifier and corresponding tokenizer
model = transformers.RobertaForSequenceClassification.from_pretrained("model", local_files_only=true)
tokenizer = transformers.AutoTokenizer.from_pretrained("gtfintechlab/FOMC-RoBERTa")

Python: PreTrainedTokenizerFast(name_or_path='gtfintechlab/FOMC-RoBERTa', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [20]:
scorer = transformers_interpret.SequenceClassificationExplainer(model, tokenizer, attribution_type="lig")

Python: <transformers_interpret.explainers.sequence_classification.SequenceClassificationExplainer object at 0x000001FDDBFEC760>

In [32]:
scorer(texts[1, :].sentence, index=0, internal_batch_size=1)

Python: [('<s>', 0.0), ('rem', -0.09617849663936606), ('ained', -0.29702715615956665), ('well', 0.25551209840018674), ('below', -0.4015987185377847), ('their', -0.013975036770146217), ('levels', -0.13794157805498133), ('at', -0.002103001077563767), ('the', 0.14926277592717438), ('beginning', 0.08228264530334284), ('of', 0.08006335674570937), ('the', 0.24284418735078794), ('year', 0.20003154332566433), (',', 0.13104142887853437), ('and', 0.17795123590947837), ('that', 0.177958452206897), ('weaker', 0.07827504560711215), ('demand', 0.06225654464144761), ('and', 0.20338402474663345), ('earlier', -0.2643532357406369), ('declines', 0.14990056209456956), ('in', -0.3258371366392156), ('oil', -0.06976564966318043), ('prices', 0.011989646035101107), ('had', -0.15355389676950998), ('been', -0.10492307025848874), ('holding', -0.07771222018418246), ('down', -0.024919108109520634), ('consumer', 0.09788758630638592), ('price', 0.011979387105458176), ('inflation', -0.09257150340664654), ('.', 0.15492

### Mask the word attributions

In [8]:
cls = TrillionDollarWords.load_model(; output_hidden_states=true)

┌ Warning: fuse_unk is unsupported, the tokenization result might be slightly different in some cases.
└ @ Transformers.HuggingFace C:\Users\drobi\.julia\packages\Transformers\lD5nW\src\huggingface\tokenizer\utils.jl:42
┌ Warning: match token `<mask>` require to match with space on either side but that is not implemented here, the tokenization result might be slightly different in some cases.
└ @ Transformers.HuggingFace C:\Users\drobi\.julia\packages\Transformers\lD5nW\src\huggingface\tokenizer\utils.jl:42


BaselineModel(GPT2TextEncoder(
├─ TextTokenizer(MatchTokenization(CodeNormalizer(BPETokenization(GPT2Tokenization, bpe = CachedBPE(BPE(50000 merges))), codemap = CodeMap{UInt8 => UInt16}(3 code-ranges)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 50265, unk = <unk>, unki = 4),
├─ codemap = CodeMap{UInt8 => UInt16}(3 code-ranges),
├─ startsym = <s>,
├─ endsym = </s>,
├─ padsym = <pad>,
├─ trunc = 256,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}(<s>:<type=1> Input:<type=1> </s>:<type=1> (</s>:<type=1> Input:<type=1> </s>:<type=1>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(256))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_or_pad(256, <pad>, tail, tail)(target.token)
  ╰─ target[token] := TextEncode

In [9]:
toks = decode(cls.tkr, encode(cls.tkr, texts[1, :].sentence).token)

256-element Vector{String}:
 "<s>"
 "rem"
 "ained"
 " well"
 " below"
 " their"
 " levels"
 " at"
 " the"
 " beginning"
 " of"
 " the"
 " year"
 ⋮
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"
 "<pad>"

In [16]:
decoded = Vector{Char}()
for token in toks
    token = string(token)
    if startswith(token, "<")
        continue
    else
        if startswith(token, " ")
            append!(decoded, token)
        else
            if length(decoded) == 0
                append!(decoded, token)
            else
                last = pop!(decoded)
                new_token = last * token
                append!(decoded, new_token)
            end
        end
    end
end

merge_truncated_tokens (generic function with 2 methods)

In [11]:
String(decoded)

"remained well below their levels at the beginning of the year, and that weaker demand and earlier declines in oil prices had been holding down consumer price inflation."

### Fill in masks

In [12]:
bert_enc = hgf"bert-base-uncased:tokenizer"
bert_model = hgf"bert-base-uncased:ForMaskedLM";

In [30]:
out = bert_model(encode(bert_enc, ["hello [MASK] world [MASK] [MASK] and my [MASK]!", "bonjour mes [MASK]."]))

(hidden_state = Float32[0.019434169 -0.006751724 … -0.3122035 0.96587026; 0.28009748 0.6731811 … 0.13828917 0.39761153; … ; -0.123422645 -0.15654448 … 0.09904317 -0.6344952; -0.05732409 -0.19550925 … 0.006869527 -0.23544282;;; -0.33048484 -0.22132668 … -0.24672607 -0.2684891; 0.046722244 -0.2989306 … 0.27542412 0.28430456; … ; 0.08061004 -0.17982382 … 0.35699606 0.35447824; 0.06478204 0.4334174 … 0.28634638 0.2961332], attention_mask = NeuralAttentionlib.LengthMask{1, Vector{Int32}}(Int32[11, 9]), logit = Float32[-6.646805 -11.58166 … -11.6885395 -12.899181; -6.594739 -11.566789 … -11.892482 -12.609668; … ; -5.804957 -10.092522 … -10.914755 -10.116999; -3.9648962 -9.437453 … -4.6001024 -10.999382;;; -6.8107824 -10.228295 … -6.8980503 -6.914237; -6.7649107 -10.272557 … -7.056045 -7.0753527; … ; -5.953972 -7.588567 … -6.2830176 -6.3007765; -3.8673246 -5.0237412 … -3.1228065 -3.1453354])

In [31]:
out = decode(bert_enc, out.logit)

11×2 Matrix{String}:
 "."        "."
 "hello"    "bon"
 ","        "##jou"
 "world"    "##s"
 ","        "me"
 "me"       "##s"
 "and"      "##nant"
 "my"       "."
 "friends"  "."
 "!"        "de"
 "."        "de"

In [56]:
function merge_truncated_words(tokens, in_word="##", between_word="", separators=[".", "de"])
    decoded = Vector{Char}()
    for token in tokens
        token = string(token)
        if token in separators
            continue
        else
            if startswith(token, in_word)
                if length(decoded) == 0
                    append!(decoded, token)
                else
                    last = pop!(decoded)
                    new_token = last * chop(token, head=2, tail=0)
                    append!(decoded, new_token)
                end
            else
                append!(decoded, " " * token)
            end
        end
    end
    return decoded
end

merge_truncated_words (generic function with 4 methods)

In [57]:
String(merge_truncated_words(out))

" hello , world , me and my friends ! bonjous mesnant"